# **Polished (sort of?) Code for Case 1**
        

In [1]:
import numpy as np 
import pandas as pd 
import os
import time

from math import floor 

for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

print('Numpy Version: ',np.__version__) 
print('Pandas Version: ',pd.__version__) 

/kaggle/input/market-basket-id-ndsc-2020/rules.csv
/kaggle/input/market-basket-id-ndsc-2020/association_order.csv
Numpy Version:  1.18.5
Pandas Version:  1.1.4


In [2]:
transaction = pd.read_csv('/kaggle/input/market-basket-id-ndsc-2020/association_order.csv')
rules = pd.read_csv('/kaggle/input/market-basket-id-ndsc-2020/rules.csv')

**Construct Dictionary for (itemid, number_of_transactions) Pair**

In [3]:
df = transaction
df.set_index(['itemid','orderid'], inplace=True)
df.sort_index(inplace=True)
dict_amount = df.reset_index().groupby('itemid')['orderid'].nunique().to_dict()

**Subset Rules DataFrame for Case 1 and Separate based on Itemid**

In [4]:
case1 = rules[~(rules['rule'].str.contains("&"))].reset_index()

case1['A'], case1['B'] = case1['rule'].str.split('>').str

case1[['A','B']] = case1[['A','B']].apply(pd.to_numeric, errors='coerce', axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


**Construct Dictionary with (itemid,orderid) Pair**

In [5]:
dict_product = df.reset_index().groupby('itemid')['orderid'].apply(list).to_dict()

**Construct Submission DataFrame for Case 1**

In [6]:
submission_case1 = pd.DataFrame(case1['rule'])
submission_case1['confidence'] = '' 

**Calculate Confidence for Case 1**

In [7]:
# Method 1: Using For-Loop
for i in range(len(submission_case1)):
    denominator = dict_amount[case1['A'][i]] 
    numerator = len(list(set.intersection(set(dict_product[case1['A'][i]]), set(dict_product[case1['B'][i]]))))
    submission_case1['confidence'][i] = floor((numerator/denominator)*1000)

In [8]:
# Method 2: Using List Comprehensions. Less readable for me :p
submission_case1['confidence'] = pd.Series([floor((len(list(set.intersection(set(dict_product[case1['A'][i]]), set(dict_product[case1['B'][i]]))))/dict_amount[case1['A'][i]])*1000) for i in range(len(submission_case1))])

**Check out Submission DataFrame for Case 1**

In [9]:
submission_case1

,rule,confidence
0,100242812>80361758,470
1,100242812>89031406,352
2,1003153762>1016449477,388
3,1006024995>2727415265,171
4,1006024995>866012366,171
...,...,...
7267,995073047>3202007524,102
7268,995073047>651958908,130
7269,995073047>7902698606,65
7270,995073047>922394800,56


# **Polished (sort of?) Code for Case 2**

**Subset for Case 2 and Case 3**

In [10]:
case23 = rules[(rules['rule'].str.contains("&"))].reset_index()

case23['LEFT'], case23['RIGHT'] = case23['rule'].str.split('>').str

case23
#case23[['LEFT','RIGHT']] = case23[['A','B']].apply(pd.to_numeric, errors='coerce', axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


,index,rule,LEFT,RIGHT
0,8,1008435393&1008455319>454021998,1008435393&1008455319,454021998
1,9,1008435393&1336146681>454021998,1008435393&1336146681,454021998
2,10,1008435393&1492482010>454021998,1008435393&1492482010,454021998
3,11,1008435393&1939300455>454021998,1008435393&1939300455,454021998
4,22,1008435393>454021998&1008455319,1008435393,454021998&1008455319
...,...,...,...,...
6961,14183,982727004>1512054476&2188141395,982727004,1512054476&2188141395
6962,14184,982727004>1512054476&2523757071,982727004,1512054476&2523757071
6963,14188,982727004>2188141395&2523757071,982727004,2188141395&2523757071
6964,14196,991988255&2057880096>926159335,991988255&2057880096,926159335


**Separate rule column into three itemid column**

In [11]:
case2 = pd.concat([case23['LEFT'], case23['RIGHT']], axis=1)
case2 = case2[(case2['LEFT'].str.contains("&"))].reset_index()
ref_case2 = case2.copy()
ref_case2.rename(columns={"LEFT": "rule"}, errors="raise",inplace = True)
ref_case2['rule'] = ref_case2['rule'] + '>' + ref_case2['RIGHT']
case2['LEFT'], case2['MIDDLE'] = case2['LEFT'].str.split('&').str

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


In [12]:
case2[['LEFT','MIDDLE','RIGHT']] = case2[['LEFT','MIDDLE','RIGHT']].apply(pd.to_numeric, errors='coerce', axis=1)

**Prepare Submission DataFrame for Case 2**

In [13]:
submission_case2 = pd.DataFrame(ref_case2['rule'])
submission_case2['confidence'] = ''

**Calculate Confidence for Case 2**

In [14]:
# Method 1: Using For-Loop
for i in range(len(submission_case2)):
    denominator = len(list(set.intersection(set(dict_product[case2['LEFT'][i]]), set(dict_product[case2['MIDDLE'][i]]))))
    numerator = len(list(set.intersection(set(dict_product[case2['LEFT'][i]]), set(dict_product[case2['MIDDLE'][i]]), set(dict_product[case2['RIGHT'][i]]))))
    submission_case2['confidence'][i] = floor((numerator/denominator)*1000)
submission_case2['confidence'] = submission_case2['confidence'].astype(int) 

**Checkout Submission DataFrame for Case 2**

In [15]:
submission_case2

,rule,confidence
0,1008435393&1008455319>454021998,352
1,1008435393&1336146681>454021998,818
2,1008435393&1492482010>454021998,875
3,1008435393&1939300455>454021998,818
4,1019107054&1697382283>684827047,750
...,...,...
3478,982727004&2188141395>1512054476,375
3479,982727004&2188141395>2523757071,375
3480,982727004&2523757071>1512054476,750
3481,982727004&2523757071>2188141395,750


# **Polished (sort of?) Code for Case 3**

**Separate Rule Column into Three Columns**

In [16]:
# hint: ubah case2 menjadi case3
case3 = pd.concat([case23['LEFT'], case23['RIGHT']], axis=1)
case3 = case3[(case3['RIGHT'].str.contains("&"))].reset_index()
ref_case3 = case3.copy()
ref_case3.rename(columns={"RIGHT": "rule"}, errors="raise",inplace = True)
ref_case3['rule'] = ref_case3['LEFT'] + '>' + ref_case3['rule']
case3['MIDDLE'], case3['RIGHT'] = case3['RIGHT'].str.split('&').str

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  import sys


In [17]:
case3[['LEFT','MIDDLE','RIGHT']] = case3[['LEFT','MIDDLE','RIGHT']].apply(pd.to_numeric, errors='coerce', axis=1)

**Prepare Submission DataFrame for Case 3**

In [18]:
submission_case3 = pd.DataFrame(ref_case3['rule'])
submission_case3['confidence'] = ''

**Calculate Confidence for Case 3**

In [19]:
# Method 1: Using For-Loop
for i in range(len(submission_case3)):
    denominator = len(list(set.intersection(set(dict_product[case3['RIGHT'][i]]), set(dict_product[case3['MIDDLE'][i]]))))
    numerator = len(list(set.intersection(set(dict_product[case3['LEFT'][i]]), set(dict_product[case3['MIDDLE'][i]]), set(dict_product[case3['RIGHT'][i]]))))
    submission_case3['confidence'][i] = floor((numerator/denominator)*1000)
submission_case3['confidence'] = submission_case3['confidence'].astype(int) 

**Checkout Submission DataFrame for Case 3**

In [20]:
submission_case3

,rule,confidence
0,1008435393>454021998&1008455319,352
1,1008435393>454021998&1336146681,562
2,1008435393>454021998&1492482010,700
3,1008435393>454021998&1939300455,450
4,1008435393>454021998&454021999,291
...,...,...
3478,959273129>959269904&1167299260,500
3479,982727004>1512054476&2188141395,750
3480,982727004>1512054476&2523757071,1000
3481,982727004>2188141395&2523757071,1000


# **Assemble DataFrame from All Cases**
        

In [21]:
submission_df = pd.concat([submission_case1,submission_case2,submission_case3])
submission_df = submission_df.reset_index().drop(['index'], axis=1).set_index('rule').loc[rules['rule']].reset_index()
submission_df.dtypes

rule          object
confidence     int64
dtype: object

In [22]:
submission_df

,rule,confidence
0,100242812>80361758,470
1,100242812>89031406,352
2,1003153762>1016449477,388
3,1006024995>2727415265,171
4,1006024995>866012366,171
...,...,...
14233,995073047>3202007524,102
14234,995073047>651958908,130
14235,995073047>7902698606,65
14236,995073047>922394800,56


# **Convert Submission DataFrame into CSV Format**

In [ ]:
submission_df.to_csv("challenge2_submission.csv",index=False)